In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re # for regex
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score
import pickle

In [2]:
data = pd.read_csv('./IMDB-Dataset.csv')
print(data.shape)
data.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [4]:
data.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [5]:
data.sentiment.replace('positive',1,inplace=True)
data.sentiment.replace('negative',0,inplace=True)
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
5,"Probably my all-time favorite movie, a story o...",1
6,I sure would like to see a resurrection of a u...,1
7,"This show was an amazing, fresh & innovative i...",0
8,Encouraged by the positive comments about this...,0
9,If you like original gut wrenching laughter yo...,1


In [6]:
def clean(text):
    cleaned = re.compile(r'<.*?>')
    return re.sub(cleaned,'',text)

data.review = data.review.apply(clean)
data.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [7]:
def is_special(text):
    rem = ''
    for i in text:
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
    return rem

data.review = data.review.apply(is_special)
data.review[0]

'One of the other reviewers has mentioned that after watching just 1 Oz episode you ll be hooked  They are right  as this is exactly what happened with me The first thing that struck me about Oz was its brutality and unflinching scenes of violence  which set in right from the word GO  Trust me  this is not a show for the faint hearted or timid  This show pulls no punches with regards to drugs  sex or violence  Its is hardcore  in the classic use of the word It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary  It focuses mainly on Emerald City  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  Em City is home to many  Aryans  Muslims  gangstas  Latinos  Christians  Italians  Irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [8]:
def to_lower(text):
    return text.lower()

data.review = data.review.apply(to_lower)
data.review[0]

'one of the other reviewers has mentioned that after watching just 1 oz episode you ll be hooked  they are right  as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence  which set in right from the word go  trust me  this is not a show for the faint hearted or timid  this show pulls no punches with regards to drugs  sex or violence  its is hardcore  in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [15]:
def rem_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    return [w for w in words if w not in stop_words]

data.review = data.review.apply(rem_stopwords)
data.review[0]

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\nayak/nltk_data'
    - 'c:\\Users\\nayak\\AppData\\Local\\Programs\\Python\\Python310\\nltk_data'
    - 'c:\\Users\\nayak\\AppData\\Local\\Programs\\Python\\Python310\\share\\nltk_data'
    - 'c:\\Users\\nayak\\AppData\\Local\\Programs\\Python\\Python310\\lib\\nltk_data'
    - 'C:\\Users\\nayak\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [14]:
def stem_txt(text):
    ss = SnowballStemmer('english')
    return " ".join([ss.stem(w) for w in text])

data.review = data.review.apply(stem_txt)
data.review[0]

'o n e   o f   t h e   o t h e r   r e v i e w e r s   h a s   m e n t i o n e d   t h a t   a f t e r   w a t c h i n g   j u s t   1   o z   e p i s o d e   y o u   l l   b e   h o o k e d     t h e y   a r e   r i g h t     a s   t h i s   i s   e x a c t l y   w h a t   h a p p e n e d   w i t h   m e   t h e   f i r s t   t h i n g   t h a t   s t r u c k   m e   a b o u t   o z   w a s   i t s   b r u t a l i t y   a n d   u n f l i n c h i n g   s c e n e s   o f   v i o l e n c e     w h i c h   s e t   i n   r i g h t   f r o m   t h e   w o r d   g o     t r u s t   m e     t h i s   i s   n o t   a   s h o w   f o r   t h e   f a i n t   h e a r t e d   o r   t i m i d     t h i s   s h o w   p u l l s   n o   p u n c h e s   w i t h   r e g a r d s   t o   d r u g s     s e x   o r   v i o l e n c e     i t s   i s   h a r d c o r e     i n   t h e   c l a s s i c   u s e   o f   t h e   w o r d   i t   i s   c a l l e d   o z   a s   t h a t   i s   t h e   n i c k n a m e

In [12]:
X = np.array(data.iloc[:,0].values)
y = np.array(data.sentiment.values)
cv = CountVectorizer(max_features = 1000)
X = cv.fit_transform(data.review).toarray()
print("X.shape = ",X.shape)
print("y.shape = ",y.shape)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [13]:
print(X)

['o n e   o f   t h e   o t h e r   r e v i e w e r s   h a s   m e n t i o n e d   t h a t   a f t e r   w a t c h i n g   j u s t   1   o z   e p i s o d e   y o u   l l   b e   h o o k e d     t h e y   a r e   r i g h t     a s   t h i s   i s   e x a c t l y   w h a t   h a p p e n e d   w i t h   m e   t h e   f i r s t   t h i n g   t h a t   s t r u c k   m e   a b o u t   o z   w a s   i t s   b r u t a l i t y   a n d   u n f l i n c h i n g   s c e n e s   o f   v i o l e n c e     w h i c h   s e t   i n   r i g h t   f r o m   t h e   w o r d   g o     t r u s t   m e     t h i s   i s   n o t   a   s h o w   f o r   t h e   f a i n t   h e a r t e d   o r   t i m i d     t h i s   s h o w   p u l l s   n o   p u n c h e s   w i t h   r e g a r d s   t o   d r u g s     s e x   o r   v i o l e n c e     i t s   i s   h a r d c o r e     i n   t h e   c l a s s i c   u s e   o f   t h e   w o r d   i t   i s   c a l l e d   o z   a s   t h a t   i s   t h e   n i c k n a m 

In [14]:
trainx,testx,trainy,testy = train_test_split(X,y,test_size=0.2,random_state=9)
print("Train shapes : X = {}, y = {}".format(trainx.shape,trainy.shape))
print("Test shapes : X = {}, y = {}".format(testx.shape,testy.shape))

Train shapes : X = (40000,), y = (40000,)
Test shapes : X = (10000,), y = (10000,)


In [15]:
gnb,mnb,bnb = GaussianNB(),MultinomialNB(alpha=1.0,fit_prior=True),BernoulliNB(alpha=1.0,fit_prior=True)
gnb.fit(trainx,trainy)
mnb.fit(trainx,trainy)
bnb.fit(trainx,trainy)

ValueError: could not convert string to float: 't h e   l a c k   o f   c o n t e n t   i n   t h i s   m o v i e   a m a z e d   m e   t h e   m o s t     f i r s t   i   t h o u g h   t h a t   p e o p l e   w a s   g o i n g   t o   c o m p a r e   t h i s   t o   r o c k   o n     b u t   i   m   r e a l l y   s u r p r i s e d   m y s e l f   t o   s a y   t h a t   t h i s   w a s   w o r s t   t h a n   r o c k   o n     s o   s o   s t o r y   h o r r i b l e   c a s t   a j a y   d e v g a n   j a m m i n g   w i t h   s a l m a n   k h a n   a n d   a s i n   y o u   g o t t a   b e   k i d d i n g   m e     t h e   m u s i c   w a s   o k a y   k h a n a b a d o s h   w a s   t h e   t r a c k   o f   t h e   m o v i e   t h e   r e s t   w a s   b a d     v i p u l   s h a h   h a s n   t   s t i l l   l e a r n   f r o m   s i n g h   i s   k i n g   s   c r i t i c a l l y   b a s h e d   c o m e d y     n o w   a s i n       w h e r e   d o   s h e   c o m e   f r o m   

In [16]:
ypg = gnb.predict(testx)
ypm = mnb.predict(testx)
ypb = bnb.predict(testx)

print("Gaussian = ",accuracy_score(testy,ypg))
print("Multinomial = ",accuracy_score(testy,ypm))
print("Bernoulli = ",accuracy_score(testy,ypb))

ValueError: could not convert string to float: 's o m e   p e o p l e   s e e m   t o   t h i n k   t h i s   w a s   t h e   w o r s t   m o v i e   t h e y   h a v e   e v e r   s e e n     a n d   i   u n d e r s t a n d   w h e r e   t h e y   r e   c o m i n g   f r o m     b u t   i   r e a l l y   h a v e   s e e n   w o r s e   t h a t   b e i n g   s a i d     t h e   m o v i e s   t h a t   i   c a n   r e c a l l     i e   t h e   o n e s   i   h a v e n   t   b l o c k e d   o u t     t h a t   w e r e   w o r s e   t h a n   t h i s     w e r e   s o   b a d   t h a t   t h e y   p h y s i c a l l y   p a i n e d   e v e r y   s e n s e   t h a t   w a s   i n v o l v e d   w i t h   w a t c h i n g   t h e   m o v i e     t h e   m o v i e s   t h a t   a r e   w o r s e   t h a n   w a r   g a m e s   2   a r e   t h e   o n e s   t h a t   m a k e   y o u   w a n t   t o   g o u g e   o u t   y o u r   e y e s     o r   s t a b   s h a r p   o b j e c t s   i n   y o u 

In [17]:
pickle.dump(bnb,open('model1.pkl','wb'))

In [18]:
rev =  """Terrible. Complete trash. Brainless tripe. Insulting to anyone who isn't an 8 year old fan boy. Im actually pretty disgusted that this movie is making the money it is - what does it say about the people who brainlessly hand over the hard earned cash to be 'entertained' in this fashion and then come here to leave a positive 8.8 review?? Oh yes, they are morons. Its the only sensible conclusion to draw. How anyone can rate this movie amongst the pantheon of great titles is beyond me.

So trying to find something constructive to say about this title is hard...I enjoyed Iron Man? Tony Stark is an inspirational character in his own movies but here he is a pale shadow of that...About the only 'hook' this movie had into me was wondering when and if Iron Man would knock Captain America out...Oh how I wished he had :( What were these other characters anyways? Useless, bickering idiots who really couldn't organise happy times in a brewery. The film was a chaotic mish mash of action elements and failed 'set pieces'...

I found the villain to be quite amusing.

And now I give up. This movie is not robbing any more of my time but I felt I ought to contribute to restoring the obvious fake rating and reviews this movie has been getting on IMDb."""
f1 = clean(rev)
f2 = is_special(f1)
f3 = to_lower(f2)
f4 = rem_stopwords(f3)
f5 = stem_txt(f4)

bow,words = [],word_tokenize(f5)
for word in words:
    bow.append(words.count(word))
#np.array(bow).reshape(1,3000)
#bow.shape
word_dict = cv.vocabulary_
pickle.dump(word_dict,open('bow.pkl','wb'))

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\nayak/nltk_data'
    - 'c:\\Users\\nayak\\AppData\\Local\\Programs\\Python\\Python310\\nltk_data'
    - 'c:\\Users\\nayak\\AppData\\Local\\Programs\\Python\\Python310\\share\\nltk_data'
    - 'c:\\Users\\nayak\\AppData\\Local\\Programs\\Python\\Python310\\lib\\nltk_data'
    - 'C:\\Users\\nayak\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [19]:
inp = []
for i in word_dict:
    inp.append(f5.count(i[0]))
y_pred = bnb.predict(np.array(inp).reshape(1,1000))

NameError: name 'word_dict' is not defined